# 13. 기계적 특성 계산 (수화 후)

## 목적
MatterGen이 발견한 신규 물질의 **수화 후 기계적 특성**을 계산하여  
콘크리트 재료로서의 가능성을 평가합니다.

---

### 워크플로우

```
MatterGen 구조 → 물 추가 → MD 평형화 (0.5ps) → 기계적 특성 계산
```

### 계산할 특성

| 특성 | 단위 | 의미 |
|------|------|------|
| **Bulk Modulus (K)** | GPa | 체적 압축 저항 (높을수록 단단함) |
| **Shear Modulus (G)** | GPa | 전단 저항 |
| **Young's Modulus (E)** | GPa | 인장/압축 강성 |

### 참고: 일반적인 시멘트 특성

| 재료 | Bulk Modulus | Young's Modulus |
|------|-------------:|----------------:|
| Portland Cement (수화 후) | 40-50 GPa | 20-30 GPa |
| C-S-H Gel | 20-40 GPa | 15-25 GPa |

## 1. 환경 설정

In [12]:
import sys
from pathlib import Path
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc
import torch

# 프로젝트 경로
PROJECT_ROOT = Path.cwd().parent.parent
RESULTS_DIR = PROJECT_ROOT / 'data' / 'results'
MATTERGEN_DIR = PROJECT_ROOT / 'data' / 'mattergen'

# 결과 디렉토리 생성
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project Root: {PROJECT_ROOT}")
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Project Root: c:\cement_final
GPU Available: True
GPU: NVIDIA GeForce RTX 4070 Laptop GPU


In [13]:
# CHGNet 로드
from chgnet.model import CHGNet
from chgnet.model.dynamics import CHGNetCalculator

print("Loading CHGNet...")
model = CHGNet.load()

# GPU 사용 설정
USE_DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
calc = CHGNetCalculator(model, use_device=USE_DEVICE)
print(f"CHGNet loaded (device: {USE_DEVICE})")

Loading CHGNet...
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
CHGNet will run on cuda
CHGNet loaded (device: cuda)


## 2. 구조 로딩

In [14]:
from ase.io import read

# MatterGen 유효 구조 로딩
structures = []

phase_dirs = [
    'phase1_Ca_Si_Al_O',
    'phase1_Ca_Si_Al_Fe_O', 
    'phase2_Ca_Si_O',
    'phase2_Ca_Si_Mg_O'
]

# 허용 원소
ALLOWED = {'Ca', 'Si', 'Al', 'Fe', 'Mg', 'O'}

print("Loading MatterGen structures:")
for phase in phase_dirs:
    phase_path = MATTERGEN_DIR / phase
    if not phase_path.exists():
        continue
    
    for cif in sorted(phase_path.glob('*.cif')):
        try:
            atoms = read(cif)
            elements = set(atoms.get_chemical_symbols())
            
            # 허용 원소만
            if elements.issubset(ALLOWED) and 'Ca' in elements:
                structures.append({
                    'name': f"{phase}_{cif.stem}",
                    'atoms': atoms,
                    'formula': atoms.get_chemical_formula(mode='hill'),
                    'source': 'MatterGen',
                    'n_atoms': len(atoms)
                })
        except Exception as e:
            pass

# 원자 수 기준 정렬 (작은 것부터 - 계산 빠름)
structures = sorted(structures, key=lambda x: x['n_atoms'])

print(f"Loaded: {len(structures)} valid structures")
print(f"Size range: {structures[0]['n_atoms']} - {structures[-1]['n_atoms']} atoms")

Loading MatterGen structures:
Loaded: 26 valid structures
Size range: 4 - 20 atoms


## 3. 수화 시스템 생성 함수

In [15]:
from ase import Atoms
from ase.build import make_supercell

def add_water_molecules(atoms, n_water=5, vacuum=8.0):
    """
    구조에 물 분자 추가
    
    Parameters:
    -----------
    atoms : ASE Atoms
        원본 구조
    n_water : int
        추가할 물 분자 수
    vacuum : float
        물 분자를 위한 공간 (Å)
    
    Returns:
    --------
    ASE Atoms with water
    """
    atoms_work = atoms.copy()
    
    # 셀 확장 (물 분자를 위한 공간)
    cell = atoms_work.get_cell()
    new_cell = cell.copy()
    new_cell[2, 2] += vacuum  # z 방향으로 확장
    atoms_work.set_cell(new_cell)
    
    # 물 분자 추가
    max_z = atoms_work.positions[:, 2].max()
    
    for i in range(n_water):
        # 물 분자 위치 (구조 위에 랜덤 배치)
        x = np.random.uniform(0, cell[0, 0])
        y = np.random.uniform(0, cell[1, 1])
        z = max_z + 2.5 + i * 1.5  # 2.5Å 위에서 시작
        
        # H2O 분자 (간단한 구조)
        water = Atoms('OH2',
                      positions=[
                          [x, y, z],           # O
                          [x + 0.76, y + 0.59, z],  # H1
                          [x - 0.76, y + 0.59, z]   # H2
                      ])
        atoms_work += water
    
    return atoms_work

print("Water addition function defined")

Water addition function defined


## 4. 탄성 특성 계산 함수

In [16]:
from ase.filters import ExpCellFilter
from ase.optimize import BFGS
from ase.md.langevin import Langevin
from ase import units

def run_hydration_equilibration(atoms, calc, temperature=300, timestep=1.0, n_steps=500):
    """
    짧은 MD로 수화 시스템 평형화
    
    Parameters:
    -----------
    atoms : ASE Atoms (물 추가된 상태)
    calc : Calculator
    temperature : float (K)
    timestep : float (fs)
    n_steps : int
    
    Returns:
    --------
    평형화된 ASE Atoms
    """
    atoms_work = atoms.copy()
    atoms_work.calc = calc
    
    # Langevin dynamics
    dyn = Langevin(
        atoms_work,
        timestep * units.fs,
        temperature_K=temperature,
        friction=0.02
    )
    
    # 짧은 MD 실행
    dyn.run(n_steps)
    
    return atoms_work

print("Hydration equilibration function defined")

ImportError: cannot import name 'ExpCellFilter' from 'ase.constraints' (c:\Users\ACER\anaconda3\envs\cement_final\Lib\site-packages\ase\constraints.py)

In [6]:
def calculate_bulk_modulus(atoms, calc, delta=0.01, optimize_first=True):
    """
    Bulk Modulus 계산 (유한 차분법)
    
    K = -V * (dP/dV) ≈ V * (d²E/dV²)
    
    Parameters:
    -----------
    atoms : ASE Atoms
    calc : Calculator
    delta : float
        부피 변화율 (기본 1%)
    
    Returns:
    --------
    dict with bulk_modulus, energies, volumes
    """
    atoms_work = atoms.copy()
    atoms_work.calc = calc
    
    # 먼저 구조 최적화 (선택적)
    if optimize_first:
        try:
            ecf = ExpCellFilter(atoms_work)
            opt = BFGS(ecf, logfile=None)
            opt.run(fmax=0.1, steps=30)
        except:
            pass
    
    # 기준 부피와 에너지
    V0 = atoms_work.get_volume()
    E0 = atoms_work.get_potential_energy()
    
    # 부피 변화에 따른 에너지 계산
    scales = [1 - 2*delta, 1 - delta, 1, 1 + delta, 1 + 2*delta]
    volumes = []
    energies = []
    
    for scale in scales:
        atoms_scaled = atoms_work.copy()
        atoms_scaled.calc = calc
        
        # 등방성 스케일링 (모든 방향 동일하게)
        cell = atoms_work.get_cell()
        atoms_scaled.set_cell(cell * (scale ** (1/3)), scale_atoms=True)
        
        try:
            E = atoms_scaled.get_potential_energy()
            V = atoms_scaled.get_volume()
            volumes.append(V)
            energies.append(E)
        except:
            return None
    
    # 2차 다항식 피팅: E(V) = a*V^2 + b*V + c
    try:
        coeffs = np.polyfit(volumes, energies, 2)
        a, b, c = coeffs
        
        # Bulk Modulus: K = V0 * d²E/dV² = V0 * 2a
        # 단위 변환: eV/Å³ → GPa (1 eV/Å³ = 160.2 GPa)
        K = V0 * 2 * a * 160.2
        
        return {
            'bulk_modulus_GPa': float(K),
            'V0': float(V0),
            'E0': float(E0),
            'volumes': volumes,
            'energies': energies,
            'fit_coeffs': coeffs.tolist()
        }
    except:
        return None

print("Bulk modulus function defined")

Bulk modulus function defined


In [7]:
def estimate_mechanical_properties(K):
    """
    Bulk Modulus로부터 다른 기계적 특성 추정
    (경험적 관계식 사용, 세라믹/산화물 기준)
    
    - Poisson's ratio ν ≈ 0.25 (세라믹 평균)
    - G = 3K(1-2ν) / 2(1+ν)
    - E = 9KG / (3K + G)
    """
    nu = 0.25  # Poisson's ratio (세라믹 평균)
    
    # Shear Modulus
    G = 3 * K * (1 - 2*nu) / (2 * (1 + nu))
    
    # Young's Modulus  
    E = 9 * K * G / (3 * K + G)
    
    return {
        'bulk_modulus_K': K,
        'shear_modulus_G': G,
        'youngs_modulus_E': E,
        'poissons_ratio': nu
    }

print("Property estimation function defined")

Property estimation function defined


## 5. 수화 + 기계적 특성 계산

### 파이프라인
```
구조 → 물 추가 (5개) → MD 평형화 (500 steps) → Bulk Modulus 계산
```

In [8]:
# 설정
N_WATER = 5           # 추가할 물 분자 수
MD_STEPS = 500        # 평형화 스텝 (0.5 ps)
N_CALC = min(10, len(structures))  # 계산할 구조 수

print(f"Settings:")
print(f"  - Water molecules: {N_WATER}")
print(f"  - MD equilibration: {MD_STEPS} steps")
print(f"  - Structures to calculate: {N_CALC}")

Settings:
  - Water molecules: 5
  - MD equilibration: 500 steps
  - Structures to calculate: 10


In [9]:
# 체크포인트 로딩
CHECKPOINT_PATH = RESULTS_DIR / 'mechanical_checkpoint.json'

if CHECKPOINT_PATH.exists():
    with open(CHECKPOINT_PATH) as f:
        checkpoint = json.load(f)
    mechanical_results = checkpoint.get('results', [])
    completed_names = {r['name'] for r in mechanical_results}
    print(f"Checkpoint loaded: {len(mechanical_results)} completed")
else:
    mechanical_results = []
    completed_names = set()
    print("Starting fresh")

Starting fresh


In [10]:
# 메인 계산 루프
print("\n" + "=" * 70)
print("HYDRATED MECHANICAL PROPERTIES CALCULATION")
print("=" * 70)

for i, s in enumerate(tqdm(structures[:N_CALC], desc="Overall"), 1):
    # 이미 완료된 구조 스킵
    if s['name'] in completed_names:
        print(f"[{i}/{N_CALC}] {s['formula']} - SKIP (already done)")
        continue
    
    print(f"\n[{i}/{N_CALC}] {s['formula']} ({s['n_atoms']} atoms)")
    
    try:
        # Step 1: 물 추가
        print("  Step 1: Adding water...")
        hydrated = add_water_molecules(s['atoms'], n_water=N_WATER)
        
        # Step 2: MD 평형화
        print(f"  Step 2: MD equilibration ({MD_STEPS} steps)...")
        equilibrated = run_hydration_equilibration(hydrated, calc, n_steps=MD_STEPS)
        
        # Step 3: Bulk Modulus 계산
        print("  Step 3: Calculating Bulk Modulus...")
        result = calculate_bulk_modulus(equilibrated, calc, optimize_first=False)
        
        if result and result['bulk_modulus_GPa'] > 0:
            K = result['bulk_modulus_GPa']
            props = estimate_mechanical_properties(K)
            
            mechanical_results.append({
                'name': s['name'],
                'formula': s['formula'],
                'n_atoms_original': s['n_atoms'],
                'n_atoms_hydrated': len(equilibrated),
                'n_water': N_WATER,
                'state': 'hydrated',
                **props
            })
            completed_names.add(s['name'])
            
            print(f"  Result: K = {K:.1f} GPa, E = {props['youngs_modulus_E']:.1f} GPa")
        else:
            print("  [FAILED] Bulk modulus calculation failed")
        
    except Exception as e:
        print(f"  [ERROR] {str(e)[:50]}")
    
    # 체크포인트 저장
    checkpoint_data = {'results': mechanical_results, 'settings': {'n_water': N_WATER, 'md_steps': MD_STEPS}}
    with open(CHECKPOINT_PATH, 'w') as f:
        json.dump(checkpoint_data, f, indent=2)
    
    # 메모리 정리
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print(f"\n" + "=" * 70)
print(f"COMPLETED: {len(mechanical_results)} structures")
print("=" * 70)


HYDRATED MECHANICAL PROPERTIES CALCULATION


Overall:  10%|█         | 1/10 [00:00<00:01,  5.76it/s]


[1/10] Ca2OSi (4 atoms)
  Step 1: Adding water...
  Step 2: MD equilibration (500 steps)...
  [ERROR] name 'run_hydration_equilibration' is not defined

[2/10] CaMgOSi (4 atoms)
  Step 1: Adding water...
  Step 2: MD equilibration (500 steps)...
  [ERROR] name 'run_hydration_equilibration' is not defined


Overall:  20%|██        | 2/10 [00:00<00:01,  5.82it/s]


[3/10] CaMgOSi (4 atoms)
  Step 1: Adding water...
  Step 2: MD equilibration (500 steps)...
  [ERROR] name 'run_hydration_equilibration' is not defined


...


[6/10] AlCaOSi3 (6 atoms)
  Step 1: Adding water...
  Step 2: MD equilibration (500 steps)...
  [ERROR] name 'run_hydration_equilibration' is not defined

[7/10] Al2CaO2Si (6 atoms)
  Step 1: Adding water...
  Step 2: MD equilibration (500 steps)...
  [ERROR] name 'run_hydration_equilibration' is not defined


Overall:  80%|████████  | 8/10 [00:01<00:00,  6.30it/s]


[8/10] Al2CaFe2O (6 atoms)
  Step 1: Adding water...
  Step 2: MD equilibration (500 steps)...
  [ERROR] name 'run_hydration_equilibration' is not defined

[9/10] AlCa2O4Si (8 atoms)
  Step 1: Adding water...
  Step 2: MD equilibration (500 steps)...
  [ERROR] name 'run_hydration_equilibration' is not defined


Overall: 100%|██████████| 10/10 [00:01<00:00,  6.06it/s]


[10/10] AlCa2O4Si (8 atoms)
  Step 1: Adding water...
  Step 2: MD equilibration (500 steps)...
  [ERROR] name 'run_hydration_equilibration' is not defined

COMPLETED: 0 structures


## 6. 결과 분석

In [11]:
# 결과 DataFrame
df = pd.DataFrame(mechanical_results)

if len(df) == 0:
    print("No results yet. Please run the calculation first.")
else:
    # 참조값
    K_REF_CEMENT = 45   # Portland cement (GPa)
    K_REF_CSH = 30      # C-S-H gel (GPa)
    
    print("\n" + "=" * 80)
    print("HYDRATED MECHANICAL PROPERTIES - RESULTS")
    print("=" * 80)
    print(f"\n{'Material':<30} {'K (GPa)':<12} {'E (GPa)':<12} {'G (GPa)':<12} {'Status':<10}")
    print("-" * 80)
    print(f"{'Portland Cement (Reference)':<30} {K_REF_CEMENT:<12.1f} {25:<12.1f} {18:<12.1f} {'REF'}")
    print(f"{'C-S-H Gel (Reference)':<30} {K_REF_CSH:<12.1f} {20:<12.1f} {12:<12.1f} {'REF'}")
    print("-" * 80)
    
    for _, row in df.iterrows():
        K = row['bulk_modulus_K']
        if K >= K_REF_CEMENT * 0.8:
            status = 'EXCELLENT'
        elif K >= K_REF_CSH * 0.8:
            status = 'GOOD'
        else:
            status = 'POOR'
        print(f"{row['formula']:<30} {K:<12.1f} {row['youngs_modulus_E']:<12.1f} {row['shear_modulus_G']:<12.1f} {status}")

No results yet. Please run the calculation first.


In [ ]:
# 시멘트 기준 대비 평가
if len(df) > 0:
    K_REF = 45  # Portland cement Bulk Modulus (GPa)
    K_CSH = 30  # C-S-H gel reference
    
    df['K_ratio'] = df['bulk_modulus_K'] / K_REF
    df['grade'] = df['bulk_modulus_K'].apply(
        lambda k: 'A' if k >= K_REF*0.8 else ('B' if k >= K_CSH*0.8 else 'C')
    )
    
    print("\n" + "=" * 60)
    print("SUITABILITY ASSESSMENT")
    print("=" * 60)
    
    grade_a = df[df['grade'] == 'A']
    grade_b = df[df['grade'] == 'B']
    grade_c = df[df['grade'] == 'C']
    
    print(f"\nGrade A (Cement-level): {len(grade_a)} structures")
    print(f"Grade B (C-S-H level):  {len(grade_b)} structures")
    print(f"Grade C (Below C-S-H):  {len(grade_c)} structures")
    
    if len(grade_a) > 0:
        print("\n[Grade A - Ready for Lab Testing]:")
        for _, row in grade_a.iterrows():
            print(f"  * {row['formula']}: K={row['bulk_modulus_K']:.1f} GPa ({row['K_ratio']*100:.0f}% of cement)")
    
    if len(grade_b) > 0:
        print("\n[Grade B - Potential Candidates]:")
        for _, row in grade_b.iterrows():
            print(f"  - {row['formula']}: K={row['bulk_modulus_K']:.1f} GPa")

## 7. 시각화

In [ ]:
if len(df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    K_REF = 45  # Portland cement
    K_CSH = 30  # C-S-H gel
    
    # (A) Bulk Modulus 비교 (수화 후)
    ax1 = axes[0]
    materials = df['formula'].tolist()
    K_values = df['bulk_modulus_K'].tolist()
    
    colors = ['#28A745' if k >= K_REF*0.8 else ('#FFC107' if k >= K_CSH*0.8 else '#DC3545') for k in K_values]
    bars = ax1.barh(range(len(materials)), K_values, color=colors, edgecolor='black')
    
    # 참조선
    ax1.axvline(x=K_REF, color='blue', linestyle='--', linewidth=2, label=f'Portland Cement ({K_REF} GPa)')
    ax1.axvline(x=K_CSH, color='purple', linestyle=':', linewidth=2, label=f'C-S-H Gel ({K_CSH} GPa)')
    
    ax1.set_yticks(range(len(materials)))
    ax1.set_yticklabels(materials, fontsize=9)
    ax1.set_xlabel('Bulk Modulus K (GPa)', fontsize=11)
    ax1.set_title('(A) Hydrated Bulk Modulus Comparison', fontsize=12, fontweight='bold')
    ax1.legend(loc='lower right', fontsize=9)
    ax1.invert_yaxis()
    ax1.set_xlim(0, max(K_values) * 1.2)
    
    # (B) K vs E (기계적 특성 공간)
    ax2 = axes[1]
    
    grade_colors = {'A': '#28A745', 'B': '#FFC107', 'C': '#DC3545'}
    for _, row in df.iterrows():
        ax2.scatter(row['bulk_modulus_K'], row['youngs_modulus_E'],
                   c=grade_colors[row['grade']], s=150, edgecolors='black', linewidth=1.5)
        ax2.annotate(row['formula'][:10], (row['bulk_modulus_K'], row['youngs_modulus_E']),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # 참조점
    ax2.scatter([45], [25], marker='*', s=400, c='blue', label='Portland Cement', zorder=5)
    ax2.scatter([30], [20], marker='s', s=200, c='purple', label='C-S-H Gel', zorder=5)
    
    ax2.set_xlabel('Bulk Modulus K (GPa)', fontsize=11)
    ax2.set_ylabel("Young's Modulus E (GPa)", fontsize=11)
    ax2.set_title('(B) Mechanical Properties Space', fontsize=12, fontweight='bold')
    ax2.legend(loc='lower right', fontsize=9)
    ax2.grid(True, alpha=0.3)
    
    # 범례 추가
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#28A745', edgecolor='black', label='Grade A (Cement-level)'),
        Patch(facecolor='#FFC107', edgecolor='black', label='Grade B (C-S-H level)'),
        Patch(facecolor='#DC3545', edgecolor='black', label='Grade C (Below threshold)')
    ]
    ax2.legend(handles=legend_elements + ax2.get_legend_handles_labels()[0][-2:], loc='lower right', fontsize=8)
    
    plt.tight_layout()
    
    # Figure 저장
    fig_dir = PROJECT_ROOT / 'figures'
    fig_dir.mkdir(exist_ok=True)
    plt.savefig(fig_dir / 'mechanical_properties_hydrated.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nFigure saved: figures/mechanical_properties_hydrated.png")
else:
    print("No data to visualize.")

## 8. 결과 저장

In [ ]:
# JSON 저장
def convert_numpy(obj):
    if isinstance(obj, (np.floating, np.float64)):
        return float(obj)
    if isinstance(obj, (np.integer, np.int64)):
        return int(obj)
    if isinstance(obj, (np.bool_, bool)):
        return bool(obj)
    if isinstance(obj, dict):
        return {k: convert_numpy(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [convert_numpy(i) for i in obj]
    return obj

if len(df) > 0:
    grade_a = df[df['grade'] == 'A']
    grade_b = df[df['grade'] == 'B']
    
    results_export = {
        'methodology': {
            'description': 'Hydrated mechanical properties calculation',
            'workflow': 'MatterGen structure -> Add water -> MD equilibration -> Bulk modulus calculation',
            'water_molecules_added': N_WATER,
            'md_equilibration_steps': MD_STEPS
        },
        'references': {
            'portland_cement': {'bulk_modulus_GPa': 45, 'youngs_modulus_GPa': 25},
            'csh_gel': {'bulk_modulus_GPa': 30, 'youngs_modulus_GPa': 20}
        },
        'grading_criteria': {
            'A': 'K >= 36 GPa (cement-level)',
            'B': 'K >= 24 GPa (C-S-H level)',
            'C': 'K < 24 GPa (below threshold)'
        },
        'results': convert_numpy(df.to_dict('records')),
        'summary': {
            'total_calculated': len(df),
            'grade_A_count': len(grade_a),
            'grade_B_count': len(grade_b),
            'grade_C_count': len(df) - len(grade_a) - len(grade_b),
            'best_candidate': df.loc[df['bulk_modulus_K'].idxmax(), 'formula'] if len(df) > 0 else None,
            'best_K_GPa': float(df['bulk_modulus_K'].max()) if len(df) > 0 else None,
            'lab_ready_candidates': grade_a['formula'].tolist()
        }
    }
    
    results_path = RESULTS_DIR / 'mechanical_properties_hydrated.json'
    with open(results_path, 'w') as f:
        json.dump(results_export, f, indent=2)
    
    print(f"Results saved: {results_path}")
else:
    print("No results to save.")

In [ ]:
# CSV 저장
if len(df) > 0:
    csv_path = RESULTS_DIR / 'mechanical_properties_hydrated.csv'
    df.to_csv(csv_path, index=False)
    print(f"CSV saved: {csv_path}")
    
    # 최종 요약 출력
    print("\n" + "=" * 70)
    print("FINAL SUMMARY: LAB-READY CANDIDATES")
    print("=" * 70)
    
    if len(grade_a) > 0:
        print("\nRecommended for experimental synthesis:")
        for i, (_, row) in enumerate(grade_a.iterrows(), 1):
            print(f"  {i}. {row['formula']}")
            print(f"     - Bulk Modulus: {row['bulk_modulus_K']:.1f} GPa ({row['K_ratio']*100:.0f}% of cement)")
            print(f"     - Young's Modulus: {row['youngs_modulus_E']:.1f} GPa")
            print(f"     - Grade: A (Cement-level)")
    else:
        print("\nNo Grade A candidates found.")
        if len(grade_b) > 0:
            print(f"\n{len(grade_b)} Grade B candidates available for further investigation.")

---

## 결론

### 기계적 특성 평가 기준 (수화 후)

| 등급 | Bulk Modulus (K) | 의미 | 실험실 권장 |
|:----:|:----------------:|------|:-----------:|
| **A** | K ≥ 36 GPa | 시멘트 수준 | **즉시 실험** |
| **B** | 24 ≤ K < 36 GPa | C-S-H 수준 | 추가 검토 |
| **C** | K < 24 GPa | 기준 미달 | 제외 |

### 실험실 이행 체크리스트

Grade A 물질에 대해:

1. **재료 합성**
   - [ ] 화학 조성 확인
   - [ ] 합성 경로 설계
   - [ ] 실험실 합성 시도

2. **물성 측정**
   - [ ] XRD (결정 구조 확인)
   - [ ] 압축강도 테스트
   - [ ] 내구성 평가

3. **응용성 평가**
   - [ ] 시멘트 대체율 결정
   - [ ] 혼합 성능 테스트
   - [ ] CO₂ 저감 효과 정량화